# Consultas Integradas

Este notebook tiene el fin de generar las consultas planteadas en el trabajo práctico integrador

#### Importo librerias

In [183]:
from pathlib import Path
from db_connections import client,db_neo4j,db_redis
from src import mongo,neo4j,utils,redis
from pymongo.errors import ConnectionFailure
import os
import pandas as pd

In [197]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

✅ Conexión a MongoDB verificada.
✅ Conexión a Neo4j verificada.
✅ Conexión a Redis verificada.


###### A.Mostrar los usuarios que visitaron “Bariloche”. 

In [227]:
query = """ 
MATCH (U:Usuario)-[:VISITO]->(D:Destino)
WHERE D.ciudad='Mendoza'
RETURN U.usuario_id AS id, U.nombre AS Nombre, U.apellido AS Apellido
"""

usuarios = neo4j.consulta(db_neo4j, query)
usuarios

,id,Nombre,Apellido
0,307,Manu,Jordán
1,331,Eli,Suárez
2,4,Cleto,Palomar
3,409,Cebrián,Roca
4,350,Candelario,Ballester
5,182,Ruth,Pina
6,48,Fernanda,Romero
7,205,Jesús,Benitez
8,383,Haydée,Prats
9,272,Olalla,Barba


###### B.Mostrar los amigos de Juan que visitaron algún destino que visitó él, mostrar el nombre del Usuario y el destino. 

In [ ]:
query = query = """ 
MATCH (juan:Usuario {nombre: "Juan"})-[:AMIGO_DE]-(amigo:Usuario)
MATCH (juan)-[:VISITO]->(d:Destino)<-[:VISITO]-(amigo)
RETURN 
    amigo.nombre AS Amigo,
    d.ciudad AS Destino
ORDER BY Amigo, Destino

"""
usuarios = neo4j.consulta(db_neo4j, query)
usuarios

""


###### C.Sugerir destinos a un usuario que no haya visitado él ni sus amigos. 

###### d. Recomendar destinos basados en viajes de amigos.

###### e. Listar los hoteles en los destinos recomendados del punto anterior. 

###### f.Ver las reservas en proceso, es decir que aún no están concretadas. 

###### g. Listar los usuarios conectados actualmente. 

###### h. Mostrar los destinos con precio inferior a $100.000

###### i. Mostrar todos los Hoteles de “Jujuy”.

###### j. Mostrar la cantidad de hoteles de un destino que guste. 

###### k. Mostrar las actividades de “Ushuaia” del tipo “aventura”.

###### l. Mostrar la cantidad de reservas concretadas de cada usuario. Mostrar el usuario y la cantidad 

#### Estadísticas

###### i.Destino más visitado

###### ii.Hotel más barato

###### iii.Actividad más popular.

#### Modificaciones

###### a.Incrementar el precio de las actividades de Tucuman en 5% 

###### b. Agregar al hotel id=1 el servicio de SPA 

###### c. Eliminar el destino que desee

###### d. Eliminar un usuario que desee 

###### e. Eliminar las relaciones AMIGO_DE para un usuario que quiera. 